In [45]:
import pandas as pd
import pickle
import numpy as np
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [2]:
path = "data/"

In [3]:
def save_obj(objname, fname):
    pickle.dump( objname, open(fname, "wb" ) )
def load_obj(fname):
        return pickle.load( open(fname, "rb" ) )

In [4]:
def ReadCSV(datafile, labelfile):
    inputdata = pd.io.parsers.read_csv(open(datafile, "r"),delimiter=",")
    data = inputdata.as_matrix()
    #data = (tmpdata/255.0)-0.5
    label = np.loadtxt(open(labelfile, "rb"),delimiter=",")
    return data, label

In [110]:
class LSTMModel(object):
    def __init__(self, name):
        self.name = name
        self.model = None
        
    def compile(self, input_dim, latent_dim, num_class):
        '''Create model

        Args:
            input_dim (int): dim of embedding vector (glove dimension)
            latent_dim (int): dim of output from LSTM layer
            num_class (int): number output class
        '''
        inputs = Input(shape=(None, input_dim))
        lstm = LSTM(latent_dim, dropout=0.2)(inputs)
        out = Dense(num_class, activation='softmax')(lstm)

        self.model = Model(inputs, out)

        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.model.summary()
    
    def fit(self, train_data, val_data, batch_size=128, epochs=1, verbose=1):    
        X_train = train_data[0]
        y_train = train_data[1]

        print("Training size:", X_train.shape[0])
        print("Validation size:", val_data[0].shape[0])
        checkpointer = ModelCheckpoint(filepath=self.name+'.h5', verbose=verbose, save_best_only=False)
        self.model.fit(X_train, y_train, 
                       validation_data=val_data,
                       callbacks=[checkpointer],
                       epochs=epochs, batch_size=batch_size, verbose=verbose)
        
    def evaluate(self, X_test, y_test, batch_size=128):
        return self.model.evaluate(X_test, y_test, batch_size=batch_size)
        
    def predict(self, X, batch_size=128):
        return self.model.predict(X, batch_size=batch_size)

In [111]:
def one_hot_encoding(y):
    y = to_categorical(y)
    return y[:,1:] #remove extra zero column at the first

In [112]:
## main function
def run():
    # Step1 : reading the data and the labels
    train_data, train_label= ReadCSV(path + 'train_data.csv', path + 'train_label.csv') 
    dev_data, dev_label= ReadCSV(path + 'dev_data.csv', path + 'dev_label.csv') 
    test_data, test_label= ReadCSV(path + 'test_data.csv', path + 'test_label.csv')
    print("Train:", train_data.shape, train_label.shape)
    print("Val:", dev_data.shape, dev_label.shape)
    print("Test:", test_data.shape, test_label.shape)
    
    labels = np.concatenate((train_label, dev_label, test_label))
    number_classes = len(np.unique(labels))
    print("Number class:", number_classes)
    y_oh = one_hot_encoding(labels)
    
    train_label = y_oh[:train_label.shape[0]]
    dev_label = y_oh[train_label.shape[0]:train_label.shape[0] + dev_label.shape[0]]
    test_label = y_oh[-test_label.shape[0]:]
    
    print("One-hot encoded:", train_label.shape, dev_label.shape, test_label.shape)
    
    # Parameter to run model
    input_dim = 300
    latten_dim = 128
    
    batch_size=128 #
    epochs = 1
    
    model = LSTMModel('twitter-emotion')
    model.compile(input_dim, latten_dim, number_classes)
    
    # ready to train
    #model.fit((train_data, train_label), (dev_data, dev_label), batch_size=batch_size, epochs=epochs)
    
    # test
    model.evaluate(test_data, test_label)

In [113]:
run()

Train: (100, 7102) (7102,)
Val: (100, 1464) (1464,)
Test: (100, 2000) (2000,)
Number class: 4
One-hot encoded: (7102, 4) (1464, 4) (2000, 4)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, None, 300)         0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 128)               219648    
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 516       
Total params: 220,164
Trainable params: 220,164
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking input: expected input_12 to have 3 dimensions, but got array with shape (100, 2000)